## Sources:
https://github.com/maxbozza/HOG-SVMlight-trainer/blob/master/train_SVMlight.py

https://github.com/aarcosg/object-detector-svm-hog-python


### 1-) 

* “calculator.tif” dosyasındaki gri ölçekli görüntüde bulunan E harflerini ‘template.tif’ görüntüsünü kullanarak hem isabet-veya-ıska (hit-or-miss) dönüşümü hem de kalıp (template) eşleme yöntemi ile bulmaya çalışınız. 
* İsabet-veya-ıska dönüşümü için işlem öncesinde “calculator.tif” ve template.tif’ görüntülerini bir eşik değeri kullanarak ikili (binary) görüntülere dönüştürünüz. 
* Kalıp eşleme için ise normalleştirilmiş çapraz korelasyon (normalized cross correlation- NCC) formülünün değerini hesaplatarak görüntü üzerinde kalıp ile arama yapınız.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

from utils import img_read
from utils import show_hist

In [ ]:
img = img_read('../images/calculator.tif')

: 